# XPath
<font size="1">Это как люди X?</font>

Если вы раннее имели дело с BeautifulSoup или с Selenium, то наверняка в вашем коде встречались конструкции вида

<pre><code>
a = page.find_element_by_class('my_div_block_class')
b = a.find_element_by_id('inner_div_block')
c = b.find_element_by_id('field_with_content')
...
</code></pre>

Более того периодически возникает необходимость достать суб-блоки из блоков, определенных общим классом, например так:
<pre><code>
useful_blocks = page.find_elements_by_class('useful_block_class')
content = []
for block in useful_blocks:
    content.extend(block.find_elements_by_id('conten_parts')
</code></pre>


<font size="1">Неплохо было бы сократить подобный код до одной строчки, не так ли?)</font>

Однако для прохождения XML-подобных деревьев давно был придуман мощный инструмент, под названием XPath (XML Path Language). Его реализация включена в качестве метода в Webdriver в случае с Selenium, однако BS4 лишен такой полезной фичи (мб мне стать контрибьютором?). 

---

Давайте для большей наглядности пользы данного инструмента я попробую решить одинаковую задачу двумя методами:
- Посредством bs4 
- Посредством ~scrapy~ Selenium

---

Задание: необходимо с сайта [КиберЛенинки](https://cyberleninka.ru/) выгрузить наименования возможных тем научных статей

<font size="1">Да, кейс притянут зауши, но я просто не вспомнил на каком сайте мне действительно пригодился данный навык</font>

Что мы видим? Иметются ненумерованные списки `ul` класа `oecd`, `li` элементы которого содержат тэг `a`, текст которых и является искомыми данными.

In [1]:
# 1. BS4 sollution
import requests
from bs4 import BeautifulSoup

def get_themes_titles_bs4():
    url = 'https://cyberleninka.ru/'
    page = BeautifulSoup(requests.get(url).content, 'html.parser')
    containers = page.find_all('ul', attrs={'class':'oecd'})

    themes = []
    extractor = lambda e: e.text
    for themes_list in containers:
        candidates = themes_list.find_all('li',attrs={'class':''})
        for c in candidates:
            theme = c.find('a')
            if theme:
                themes.append(theme.text)
    return themes

result = get_themes_titles_bs4()
print(result[0], '...', result[-1], sep=', ')

Фундаментальная медицина, ..., СМИ (медиа) и массовые коммуникации


In [2]:
# 2. Selenium sollution
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

def get_themes_titles_selenium():
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get('https://cyberleninka.ru/')
    extractor = lambda e: e.text
    result = list(map(extractor, driver.find_elements_by_xpath("//ul[@class='oecd']/li[not(@class='letter')]/a")))
    driver.close()
    return result

result = get_themes_titles_selenium()
print(result[0], '...', result[-1], sep=', ')


Checking for win32 chromedriver:104.0.5112.79 in cache
Driver found in C:\Users\Tom\.wdm\chromedriver\104.0.5112.79\win32/chromedriver.exe
Фундаментальная медицина, ..., СМИ (медиа) и массовые коммуникации


---

Бесспорно в данной ситуации быстрее сработает `bs4`, однако удобность использования `XPath` выигрывает:
- В объеме получаемого кода
- В степени читаемости кода

Куда приятнее было-бы применять `XPath` вкупе с `requests`, что и позволяет делать библиотека `lxml`. Давайте теперь попробуем спарсить большие объемы данных и замерить скорость работы с использованием bs4 и lxml на примере [EBay](https://www.ebay.com/) =)

In [3]:
from functools import wraps
import time

# Для начала создадим функцию замера времени работы
def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f'Function {func.__name__}{args} {kwargs} Took {total_time:.4f} seconds')
        print('Returned values count:', len(result))
        print(f'Returned values slice: {result[0]}, ..., {result[-1]}')
        return result
    return timeit_wrapper

# И зададим пару констант
pages_limit = 10
query = 'notebook'
url = 'https://www.ebay.com/sch/i.html'
params = {
    '_from': 'R40',
    '_nkw': 'notebook',
    '_sacat': '0',
    '_pgn': 'Yonk!',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
    'Connection': 'keep-alive',
    'Cookie': 'dp1=bpbf/%23e000e0000000000000000064d75d40^u1p/QEBfX0BAX19AQA**66b890c0^bl/RU66b890c0^; nonsession=BAQAAAYJwc5B/AAaAAMoAIGa4kMA5MTg1MDUxMzE4MjBhOWY3MzY5ZmZkMmRmZmY5YzczNQAzAAZk111AMTQzMDAwAMsAAmL2MMgzNTm91vybBPy57ARyNPTIXwTwF3Qi; s=CgAD4ACBi93tAOTE4NTA1MTMxODIwYTlmNzM2OWZmZDJkZmZmOWM3MzUgff55; ebay=%5Ejs%3D1%5Esbf%3D%23%5E; __deba=7uuxKC2pLvstl-e8nSLIAkTSUuf7vUOuWxTdZSuuGCpatxJDSdHfG6CcSuZk3l2UXR8UW1J2_XMwSXzXGqPFGDRMxOmscjrBPBLFUJAt2Mg9dZsJb5hCpJFeIxt7VDHl1hvfZ4KJZSeNrIugNs6OlQ==; __uzma=a672bb5a-bd73-4c8d-94da-2967b040d084; __uzmb=1660298790; __uzmc=557452518216; __uzmd=1660299485; __uzme=9222; __uzmf=7f60009da4ab61-e221-471d-87d6-2b7e5dc497bc1660298790132695012-f7e52eabd9b8e17225; ak_bmsc=012E85F77B3FD7BF2D84E71DDAD1D5D4~000000000000000000000000000000~YAAQJvxzPs3UtV2CAQAAigWFkRCAy24iCM/91J7rJWhNIeY3WD/MxnyCZXkFlB/LmQTQODGrPUM4PWFGMhZk90eOSitfJLEGey9qzZHCEo8FYN7Gs61Ao0gVk+X7MfU+lfgvCkoANP+3YgsUOMqlhnAINIpPi3d6fjWjYJCb84ZzbdBua2mYpcHoP7j1hZ0+8UvakxpJgLvdT2rV7svgqRPc7A5LsUHqsp+HxyIe0fTtFOwxhugyBAPN99phPWQfRWO2z++jqUJUNf8fCBoEqN27Uaoz7v0NYVJTcibDhg0IZSKaPLC+UbRRAZYQXbmIQdadJCYjPl8VcrNljwuqP0+m+hP4BBEfFJJYbgtYmVUhjH36vF1YC+SCdRgxYT6WUVkyi+7InxA=; __ssds=2; __ssuzjsr2=a9be0cd8e; __uzmaj2=d041ac5c-0a73-4392-896f-ca80c2134443; __uzmbj2=1660298795; __uzmcj2=152912535579; __uzmdj2=1660298998; bm_sv=0432F98D354828D04F161C203C880D8B~YAAQJvxzPpA6tl2CAQAA252QkRBJPYGeTiJUiCU2KLOmEyfEjGO3FSvIc376MvES6uo0HFbAz1/a+G/IYEDM6W8mVhqJjiLXXnW7P5eUilmcG+YL33/+/r8FQBtPba92t84M12dTPArvdpd5oxgPzEXX+OmoG+RNqXz4C3t/aMO52IqPTlkG+PSrr0FskQyDQVxtkTfIxuQ3Zl4I1+czU4EOPM+sn0XnA6i14KrNwq3pfPuSSllf7eTRvAypzg0=~1; ns1=BAQAAAYJwc5B/AAaAANgASmTXXUBjNjl8NjAxXjE2NjAyOTg4NzgxOTleXjFeM3wyfDV8NHw3fDExXl5eNF4zXjEyXjEyXjJeMV4xXjBeMV4wXjFeNjQ0MjQ1OTA3NUOjT1xnU3iJCtqphkIAl+Kk7fiK; __gads=ID=22b130390032beb6:T=1660298893:S=ALNI_MbcAg9bpfzuuxJoXa3QGfZOfzAtDQ; __gpi=UID=00000abd686a6e22:T=1660298893:RT=1660298893:S=ALNI_MbKK3-jkDUaVjmvY2v0ghSU4IKbrQ; __gsas=ID=ff34d43e233278f3:T=1660298927:S=ALNI_Mbp1qCwSApJsPXJkBps_CS9X34K6A; ds2=',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
}

In [4]:
@timeit
def parse_ebay_by_bs4():
    titles = []
    for i in range(pages_limit):
        params['_pgn'] = i
        response = requests.get(url, params=params, headers=headers)
        page = BeautifulSoup(response.content, 'html.parser')
        results = page.find('ul', attrs={'class':'srp-results'})
        titles.extend(results.find_all('h3', attrs={'class': 's-item__title'}))
    return [t.text for t in titles]

titles = parse_ebay_by_bs4()

Function parse_ebay_by_bs4() {} Took 51.9378 seconds
Returned values count: 600
Returned values slice: CHUWI 14.1" ноутбука, компьютера, ПК, Windows 11 Home Pc 2.8GHz 6 ГБ 128 ГБ HD Wifi, ..., Dell Latitude 12 прочный планшета 7202 Intel M-5Y71 256 ГБ SSD 8 ГБ Touch cmra Gps


In [5]:
from lxml import etree

@timeit
def parse_ebay_by_lxml():
    titles = []
    for i in range(pages_limit):
        params['_pgn'] = i
        response = requests.get(url, params=params, headers=headers)
        tree = etree.HTML(response.text)
        xpath = '//ul[contains(@class,"srp-results")]/li/div/div/a/h3[@class="s-item__title"]/text()'
        titles_batch = tree.xpath(xpath)
        titles.extend(titles_batch)
    return titles

titles = parse_ebay_by_lxml()

Function parse_ebay_by_lxml() {} Took 19.2004 seconds
Returned values count: 600
Returned values slice: CHUWI 14.1" ноутбука, компьютера, ПК, Windows 11 Home Pc 2.8GHz 6 ГБ 128 ГБ HD Wifi, ..., Ноутбук HP 15-DA1051tu 15.6" i5-8250U 8 GBRAM DDR4 1 ТБ Hdmi Win10


Помимо того, что применение XPath ускорило программу в 2.5 раза, хотелось бы отметить, что при его написании мы прописали достаточно детальный путь нахождения искомых данных, что снижает риск спарсить излишние данные. Пример HTML-файла, в котором детализация местоположения данных играет роль вы можете найти ниже в Приложении №1.

Мне, правда, до сих пор неясна причина жизни bs4, и я не знаю каких-то специфических задач парсинга, в которых lxml был-бы бесполезен, а bs4 показывал-бы дейстительно стоящие результаты. Буду рад, если вы свяжетесь со мной по этому поводу =(

---

Основную информацию по `XPath` вы можете найти в [официальной документации](https://www.w3schools.com/xml/xpath_intro.asp), а дублировать её сюда я не вижу смысла.

---
Приложение №1.

<pre><div>
&lt;html&gt;
    ...
    &lt;h1&gt;Возможно вы имели ввиду:&lt;/h1&gt;
    &lt;ul class='srp-results search-candidates'&gt;
        &lt;li&gt;<b>&lt;h3 class="s-item__title candidate"&gt;Laptop&lt;/h3&gt;</b>&lt;/li&gt;
        &lt;li&gt;<b>&lt;h3 class="s-item__title candidate"&gt;Ultrabook&lt;/h3&gt;</b>&lt;/li&gt;
    &lt;/ul&gt;
    ...
    &lt;ul class='srp-results product-list'&gt;
        &lt;li&gt;
            &lt;div fileds...&gt;
                &lt;div fields...&gt;
                    &lt;a href='some_link'&gt;
                        <b>&lt;h3 class="s-item__title result"&gt;Phaser Notebook M5&lt;/h3&gt;</b>
                    &lt;/a&gt;
                &lt;/&gt;
            &lt;/div&gt;
        &lt;/li&gt;
        &lt;li&gt;
            &lt;div fileds...&gt;
                &lt;div fields...&gt;
                    &lt;a href='some_link'&gt;
                        <b>&lt;h3 class="s-item__title result"&gt;PineApple 2022 17 256GB&lt;/h3&gt;</b>
                    &lt;/a&gt;
                &lt;/&gt;
            &lt;/div&gt;
        &lt;/li&gt;
        ...
    &lt;/ul&gt;
    ...
&lt;/html&gt;
</pre></code>